In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ad_label = np.ones((250*6,), dtype=float)
ad_label[0:125] = 0
for k in range(35):
    ad_label[125+k] = k/35

In [5]:
ad_t0 = np.random.randint(125-50, 160+50, size=(5000,), dtype=int)

In [18]:
ad_label[slice(ad_t0, ad_t0+50, 1)]

TypeError: only integer scalar arrays can be converted to a scalar index